In [1]:
import pandas as pd
from pyqubo import Constraint, Array
import neal
from dwave.system import DWaveSampler, EmbeddingComposite, LeapHybridBQMSampler,  LeapHybridSampler
import dimod
import os
from dotenv import load_dotenv

In [2]:
# loading in the D-Wave Token
load_dotenv()
token_use = os.getenv("API_TOKEN")

In [3]:
data = pd.read_excel("data.xlsx")

In [4]:
columns = ["name", "position", "value", "total_points"]

In [5]:
data = data[columns]

In [6]:
df_use = data.sort_values("position").reset_index(drop=True)

In [7]:
for i in range(len(df_use)):
    df_use.loc[i, "variable"] = "x[" + str(i) + "]"
df_use

,name,position,value,total_points,variable
0,Benjamin White,DEF,56,78,x[0]
1,Vitalii Mykolenko,DEF,46,78,x[1]
2,Virgil van Dijk,DEF,63,80,x[2]
3,Kieran Trippier,DEF,67,102,x[3]
4,Oleksandr Zinchenko,DEF,52,85,x[4]
5,Trent Alexander-Arnold,DEF,85,99,x[5]
6,William Saliba,DEF,56,84,x[6]
7,Gabriel dos Santos Magalhães,DEF,50,76,x[7]
8,James Tarkowski,DEF,46,78,x[8]
9,Pedro Porro,DEF,59,85,x[9]


In [8]:
columns = ["variable","name", "position", "value", "total_points"]

In [9]:
total_points = df_use["total_points"].to_list()
value = df_use["value"].to_list()

In [10]:
df_use = df_use[columns]

In [11]:
lagrange = 1716
lagrange_budget = 1551
num_var = 38
slack_num = 1

In [12]:
defense=4
midfield=3
attack=3

In [13]:
x = Array.create('x', shape=num_var, vartype='BINARY')
s = Array.create('s', shape=slack_num, vartype='BINARY')

In [14]:
# objective function
h = sum(n * x for x, n in zip(x, total_points))

In [15]:
C1 = lagrange * Constraint((sum(x[n] for n in range(0, num_var)) - 11)**2,
                    label='11 players team')
C2 = lagrange * Constraint((sum(x[n] for n in range(0, 10))-defense)**2,
                    label=str(defense) + " defenders")
C3 = lagrange * Constraint((sum(x[n] for n in range(10, 21))-attack)**2,
                    label=str(attack) + " forwards")
C4 = lagrange * Constraint((sum(x[n] for n in range(21, 28))-1)**2,
                    label= "1 keeper")
C5 = lagrange * Constraint((sum(x[n] for n in range(28,  38))-midfield)**2,
                    label=str(midfield) + " midfielders")

In [16]:
C6 = lagrange_budget * Constraint((sum(n * x for x, n in zip(x, value)) + s[0] -700)**2,
                                  label="budget")

In [17]:
H = -1 * h + C1 + C2 + C3 + C4 + C5 + C6

In [18]:
model = H.compile()
qubo, offset = model.to_qubo()
bqm = model.to_bqm()

# Solve problem with QPU
api_token = token_use
sampler = LeapHybridSampler(token= api_token)
sampleset = sampler.sample(bqm,
                            label="FPL line-up optimization")

In [19]:
# Decode samples and select the best one
decoded_samples = model.decode_sampleset(sampleset)
best_sample = min(decoded_samples, key=lambda x: x.energy)

# Print to see if constraints are fulfilled
print(best_sample.constraints())

{'budget': (True, 0.0), '3 forwards': (True, 0.0), '11 players team': (True, 0.0), '1 keeper': (True, 0.0), '3 midfielders': (True, 0.0), '4 defenders': (True, 0.0)}


In [20]:
# Print results for best line-up
lineup_df = pd.DataFrame(best_sample.sample.items())
lineup_df.columns = ['variable', 'selected']
lineup_df = lineup_df[(lineup_df['variable'].str.startswith(
    'x', na=False)) & (lineup_df['selected'] == 1)]
lineup_df = df_use.merge(lineup_df, on=['variable'])

# Print line-up and maximized energy for the objective function
gk = lineup_df[lineup_df["position"] == "GK"]
defense_list = lineup_df[lineup_df["position"] == "DEF"]
midfield_list = lineup_df[lineup_df["position"] == "MID"]
attack_list = lineup_df[lineup_df["position"] == "FWD"]
ordered_lineup_df = pd.concat([gk, defense_list, midfield_list, attack_list], axis=0).reset_index(drop=True)
ordered_lineup_df = ordered_lineup_df[["name", "position", "value", "total_points"]]
print(ordered_lineup_df)
print("Total sum of points: ", ordered_lineup_df['total_points'].sum())
print("Total budget: ", ordered_lineup_df['value'].sum())

                                name position  value  total_points
0                         Bernd Leno       GK     48            82
1                  Vitalii Mykolenko      DEF     46            78
2                Oleksandr Zinchenko      DEF     52            85
3       Gabriel dos Santos Magalhães      DEF     50            76
4                        Pedro Porro      DEF     59            85
5       Douglas Luiz Soares de Paulo      MID     56           103
6                      Son Heung-min      MID     97           136
7                      Mohamed Salah      MID    130           156
8   Matheus Santos Carneiro Da Cunha      FWD     58           107
9      João Pedro Junqueira de Jesus      FWD     55            83
10                    Elijah Adebayo      FWD     48            75
Total sum of points:  1066
Total budget:  699


In [26]:
n = 4
#from collections import 

In [29]:
x = 10 - n
y = x - 1

In [30]:
for i in range(1, 10 - (x+y)):
    print(y)